In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Trabajo Final: Jime, asistente virtual de cobros\n",
        "\n",
        "## Resumen\n",
        "\n",
        "El problema elegido para resolver es la cobranza preventiva de la factura de luz a personas que no están todavía en deuda. Esto es importante porque permite a las empresas evitar pérdidas financieras y mantener relaciones positivas con sus clientes.\n",
        "\n",
        "### 1. Introducción\n",
        "\n",
        "La solución propuesta es Jime, un asistente virtual para Telegram que resuelve un caso de cobranza preventiva con personas que no están todavía en deuda. La idea es enviar un recordatorio de pago y llegar a un acuerdo con una fecha de corte futura. En el prompt indicaremos distintos pasos y escenarios asociados al pago de la deuda. Pensé hacerlo en cuatro pasos:\n",
        "\n",
        "1. Validación de identidad del usuario.\n",
        "2. Comunicación de los datos del pago y pregunta por posible acuerdo.\n",
        "3. Manejo de objeciones.\n",
        "4. Despedida y cierre de la conversación.\n",
        "\n",
        "### 2. Objetivos\n",
        "\n",
        "- Lograr compromisos de pagos adeudados en el servicio de luz.\n",
        "- Mejorar la imagen de la empresa, logrando automatizar este proceso y estableciendo un punto de contacto cordial y distendido con el usuario.\n",
        "\n",
        "### 3. Metodología\n",
        "\n",
        "1. Desarrollo del prompt: Elaboración de instrucciones precisas para lograr la operación.\n",
        "2. Conexión e integración con Telegram.\n",
        "3. Testeos manuales del asistente virtual para verificar si el prompt necesita iteración.\n",
        "4. Iteración del prompt.\n",
        "5. Nuevas pruebas y ajustes (si fueran necesarios).\n",
        "\n",
        "### 4. Herramientas y Tecnologías\n",
        "\n",
        "- Python\n",
        "- OpenAI GPT-4 API\n",
        "- Zero-shot y one-shot prompting: para indicar las acciones de manera precisa y mitigar errores de clasificación desde el prompt mediante el uso de ejemplos.\n",
        "- Integración con Telegram\n"
      ]
    }
  ]
}


In [ ]:
pip install groq

from groq import Groq

# Inicializar el cliente de Groq con tu API key
cliente = Groq(api_key="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")

In [ ]:
import os
import telebot
from groq import Groq

# Cargar credenciales desde variables de entorno
TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Validación de credenciales
if not TELEGRAM_TOKEN or not GROQ_API_KEY:
    raise ValueError("Faltan los tokens de acceso. Configura las variables de entorno correctamente.")

# Inicialización de clientes
bot = telebot.TeleBot(TELEGRAM_TOKEN)
groq_client = Groq(api_key=GROQ_API_KEY)

# Historial de conversaciones
conversation_history = {}

# Estado de conversación de cada usuario
user_states = {}

def get_groq_response(user_id, user_message):
    """
    Obtiene la respuesta de Groq basada en el historial de conversación.
    Aplica la lógica para verificar identidad y gestionar pagos.
    """
    if user_id not in conversation_history:
        conversation_history[user_id] = []

    conversation_history[user_id].append({"role": "user", "content": user_message})

    if len(conversation_history[user_id]) > 10:
        conversation_history[user_id] = conversation_history[user_id][-10:]

    # Obtener el estado actual del usuario
    user_state = user_states.get(user_id, "identificacion")

    try:
        # Llamada a Groq con instrucciones específicas
        completion = groq_client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=[
                {"role": "system", "content": (
                    "Eres Jime, un asistente de cobranza de Edesur. "
                    "Sigues este proceso: "
                    "1. Confirmar la identidad del usuario. Si no es la persona correcta, despedirse. "
                    "2. Si es la persona correcta, informar el monto adeudado y preguntar si puede pagar en la fecha límite. "
                    "3. Si acepta, agradecerle. "
                    "4. Si no puede pagar en la fecha, negociar una nueva fecha dentro del límite permitido. "
                    "5. Si rechaza pagar, despedirse amablemente."
                )},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7,
            max_tokens=1024
        )

        response = completion.choices[0].message.content

        # Manejo de estados según respuesta
        if user_state == "identificacion":
            if "no soy" in user_message.lower() or "equivocado" in response.lower():
                response = "Lo siento por la confusión. ¡Hasta luego!"
                user_states[user_id] = "finalizado"
            else:
                response += "\nTu deuda actual con Edesur es de $12,500. ¿Puedes pagar antes del 10 de marzo?"
                user_states[user_id] = "gestion_pago"

        elif user_state == "gestion_pago":
            if "sí" in user_message.lower():
                response = "Gracias por tu compromiso de pago. Recuerda que puedes hacerlo a través de tu home banking o Mercado Pago. ¡Que tengas un buen día! Hasta pronto."
                user_states[user_id] = "finalizado"
            elif "no" in user_message.lower():
                response = "Entiendo. ¿Podrías pagar antes del 15 de marzo?"
                user_states[user_id] = "nueva_fecha"
            else:
                response = "No entendí tu respuesta. ¿Podrías confirmarme si puedes pagar en la fecha indicada?"

        elif user_state == "nueva_fecha":
            if "sí" in user_message.lower():
                response = "Gracias por tu compromiso de pago. Recuerda que puedes hacerlo a través de tu home banking o Mercado Pago. ¡Que tengas un buen día! Hasta pronto."
                user_states[user_id] = "finalizado"
            elif "no" in user_message.lower():
                response = "Entiendo tu situación. En los próximos días nos contactaremos contigo para encontrar una solución. ¡Hasta luego!"
                user_states[user_id] = "finalizado"
            else:
                response = "No entendí tu respuesta. ¿Podrías confirmarme si puedes pagar en la nueva fecha?"

        conversation_history[user_id].append({"role": "assistant", "content": response})

        return response

    except Exception as e:
        return f"Lo siento, hubo un error al procesar tu solicitud: {str(e)}"

# Manejador del comando /start
@bot.message_handler(commands=['start'])
def send_welcome(message):
    user_id = message.from_user.id
    conversation_history[user_id] = []
    user_states[user_id] = "identificacion"
    bot.reply_to(message, "Hola, soy Jime, asistente virtual de Edesur. ¿Me podrías confirmar tu nombre para proceder con la consulta?")

# Manejador del comando /clear
@bot.message_handler(commands=['clear'])
def clear_history(message):
    user_id = message.from_user.id
    conversation_history[user_id] = []
    user_states[user_id] = "identificacion"
    bot.reply_to(message, "Historial de conversación borrado.")

# Manejador para todos los mensajes
@bot.message_handler(func=lambda message: True)
def handle_messages(message):
    user_id = message.from_user.id
    user_message = message.text

    response = get_groq_response(user_id, user_message)
    bot.reply_to(message, response)

def main():
    try:
        print("Jime está en línea y lista para cobrar facturas...")
        bot.polling(none_stop=True)
    except Exception as e:
        print(f"Error en el bot: {e}")
        main()

if __name__ == "__main__":
    main()
